<a href="https://colab.research.google.com/github/rkorolov/linearAndPolynomialModels/blob/main/linear_and_poly_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [ ]:

# ============================================================
# Imports
# ============================================================

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import matplotlib.pyplot as plt


### 1.1 - Load & Inspect the Dataset

Since there aren't any missing values, we don't need to do anything extra to account for them.

In [ ]:

# ============================================================
# Load dataset
# ============================================================

DATA_PATH = "FuelEconomy.csv"
df = pd.read_csv(DATA_PATH)

print("Shape:", df.shape)
print("\nColumns:")
print(df.columns.tolist())

display(df.head())

print("\nSummary statistics:")
display(df.describe(include="all"))

print("\nMissing values per column:")
display(df.isna().sum())


Shape: (100, 2)

Columns:
['Horse Power', 'Fuel Economy (MPG)']


,Horse Power,Fuel Economy (MPG)
0,118.770799,29.344195
1,176.326567,24.695934
2,219.262465,23.952010
3,187.310009,23.384546
4,218.594340,23.426739



Summary statistics:


,Horse Power,Fuel Economy (MPG)
count,100.000000,100.000000
mean,213.676190,23.178501
std,62.061726,4.701666
min,50.000000,10.000000
25%,174.996514,20.439516
50%,218.928402,23.143192
75%,251.706476,26.089933
max,350.000000,35.000000



Missing values per column:


,0
Horse Power,0
Fuel Economy (MPG),0


In [ ]:
# ============================================================
# Utility functions
# ============================================================

TARGET_COL = "Horse Power"

def prepare_xy(df_in, target_col=TARGET_COL):
    """Drop missing rows, split into X and y."""
    df_clean = df_in.dropna().copy()
    X = df_clean.drop(columns=[target_col])
    y = df_clean[target_col]
    return X, y

def split_data(X, y, test_size=0.30, random_state=42):
    """70/30 random train-test split."""
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

def compute_metrics(y_true, y_pred):
    """Return MSE, MAE, R^2."""
    return {
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "R^2": r2_score(y_true, y_pred),
    }

def _get_linear_parts(model, input_feature_names):
    """Extract (intercept, coefficients, feature_names) from either:
       - LinearRegression
       - Pipeline(PolynomialFeatures -> LinearRegression)
    """
    # Plain LinearRegression
    if isinstance(model, LinearRegression):
        intercept = float(model.intercept_)
        coefs = np.array(model.coef_).ravel()
        feat_names = np.array(list(input_feature_names))
        return intercept, coefs, feat_names

    # Polynomial pipeline
    if hasattr(model, "named_steps") and "poly" in model.named_steps and "lr" in model.named_steps:
        poly = model.named_steps["poly"]
        lr = model.named_steps["lr"]

        feat_names = poly.get_feature_names_out(input_features=list(input_feature_names))
        intercept = float(lr.intercept_)
        coefs = np.array(lr.coef_).ravel()
        return intercept, coefs, np.array(feat_names)

    raise ValueError("Unsupported model type for equation printing.")

def print_fitted_equation(model, input_feature_names, target_name=TARGET_COL, top_k_terms=15):
    """Print a readable fitted equation.

    For polynomial models, the number of terms can become very large,
    so we print only the TOP-K terms by absolute coefficient magnitude.
    """
    intercept, coefs, feat_names = _get_linear_parts(model, input_feature_names)

    # Sort by absolute coefficient magnitude
    order = np.argsort(np.abs(coefs))[::-1]
    order = order[:min(top_k_terms, len(coefs))]

    terms = []
    for idx in order:
        terms.append(f"({coefs[idx]:+.4f}) * {feat_names[idx]}")

    eq = f"{target_name} = {intercept:.4f} " + " ".join(terms)

    print("\n--- Fitted Model Equation (Top Terms) ---")
    print(eq)
    if len(coefs) > top_k_terms:
        print(f"(Showing top {top_k_terms} terms out of {len(coefs)} total terms.)")

def plot_actual_vs_predicted_test(y_test, y_pred, title, max_points=300):
    """Scatter plot of Actual vs Predicted values on the TEST set.

    We plot both series against a sample index, using:
    - Actual: blue circles
    - Predicted: red x's

    If test set is large, we randomly sample up to max_points points for readability.
    """
    y_test = np.array(y_test)
    y_pred = np.array(y_pred)

    n = len(y_test)
    if n > max_points:
        rng = np.random.default_rng(0)
        sel = rng.choice(n, size=max_points, replace=False)
        y_test = y_test[sel]
        y_pred = y_pred[sel]

    x = np.arange(len(y_test))

    plt.figure(figsize=(12, 4))
    plt.scatter(x, y_test, marker="o", alpha=0.8, label="Actual (Test)")
    plt.scatter(x, y_pred, marker="x", alpha=0.8, label="Predicted (Test)")
    plt.title(title)
    plt.xlabel("Test sample index (subset)")
    plt.ylabel(TARGET_COL)
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.legend()
    plt.show()

def run_models_and_evaluate(df_in, scenario_name, degrees=(1, 2, 3, 4),
                            target_col=TARGET_COL, test_size=0.30, random_state=42,
                            show_equation=True, show_plots=True, top_k_terms=15):
    """Train/evaluate linear (deg=1) + polynomial regression models.

    Returns a DataFrame of metrics.
    Also prints fitted equations and scatter plots (test set) for each model.
    """
    X, y = prepare_xy(df_in, target_col=target_col)
    X_train, X_test, y_train, y_test = split_data(X, y, test_size=test_size, random_state=random_state)

    rows = []

    for deg in degrees:
        if deg == 1:
            model = LinearRegression()
            model_name = "Linear Regression"
        else:
            model = Pipeline([
                ("poly", PolynomialFeatures(degree=deg, include_bias=False)),
                ("lr", LinearRegression())
            ])
            model_name = f"Polynomial Regression (degree={deg})"

        # Fit model
        model.fit(X_train, y_train)

        # Predict
        yhat_train = model.predict(X_train)
        yhat_test  = model.predict(X_test)

        # Metrics
        train_m = compute_metrics(y_train, yhat_train)
        test_m  = compute_metrics(y_test, yhat_test)

        # Report equation + plot (TEST set)
        print("\n============================================================")
        print(f"Scenario: {scenario_name}")
        print(f"Model: {model_name}")
        print("============================================================")

        if show_equation:
            print_fitted_equation(
                model=model,
                input_feature_names=X_train.columns,
                target_name=target_col,
                top_k_terms=top_k_terms
            )

        if show_plots:
            plot_actual_vs_predicted_test(
                y_test=y_test,
                y_pred=yhat_test,
                title=f"{scenario_name} — {model_name} (Test Set: Actual vs Predicted)"
            )

        rows.append({
            "Scenario": scenario_name,
            "Model": model_name,
            "Train MSE": train_m["MSE"],
            "Train MAE": train_m["MAE"],
            "Train R^2": train_m["R^2"],
            "Test MSE": test_m["MSE"],
            "Test MAE": test_m["MAE"],
            "Test R^2": test_m["R^2"],
            "Train size": len(X_train),
            "Test size": len(X_test),
        })

    return pd.DataFrame(rows)


### 1.2 - Train/Test Split

In [ ]:
# Randomly split the dataset into 70% training and 30% testing.
# Use a fixed random state for reproducibility -- seed=42.

TARGET_COL = "Horse Power"
EXTRA_COL = "Fuel Economy (MPG)"

X, y = prepare_xy(df, TARGET_COL)
x_train, x_test, y_train, y_test = split_data(X, y) # default 30% testing


### 1.3 Model training: Linear + Polynomial regression

In [ ]:
# Linear Regression
ln_model = LinearRegression()

ln_model.fit(x_train, y_train)

yhat_train_ln = ln_model.predict(x_train)
yhat_test_ln = ln_model.predict(x_test)


# Degree 2
deg_two_model = Pipeline([
                ("poly", PolynomialFeatures(degree=2, include_bias=False)),
                ("lr", LinearRegression())])

deg_two_model.fit(x_train, y_train)

yhat_train_two = deg_two_model.predict(x_train)
yhat_test_two = deg_two_model.predict(x_test)

# Degree 3
deg_three_model = Pipeline([
                ("poly", PolynomialFeatures(degree=3, include_bias=False)),
                ("lr", LinearRegression())])

deg_three_model.fit(x_train, y_train)

yhat_train_three = deg_three_model.predict(x_train)
yhat_test_three = deg_three_model.predict(x_test)

# Degree 4
deg_four_model = Pipeline([
                ("poly", PolynomialFeatures(degree=4, include_bias=False)),
                ("lr", LinearRegression())])

deg_four_model.fit(x_train, y_train)

yhat_train_four = deg_four_model.predict(x_train)
yhat_test_four = deg_four_model.predict(x_test)



In [ ]:
# Print metrics
print("Table 1: Predicting Horse Power using Fuel Economy (MPG)")
print("=================================================================================================")
print("Model             Train MSE  Train MAE          Train R^2 | Test MSE Test MAE           Test R^2")
print("=================================================================================================")
for model_name, ytr, yte in [
    ("Linear Reg      ", yhat_train_ln, yhat_test_ln),
    ("Poly Reg (deg 2)", yhat_train_two, yhat_test_two),
    ("Poly Reg (deg 3)", yhat_train_three, yhat_test_three),
    ("Poly Reg (deg 4)", yhat_train_four, yhat_test_four)
]:
        train_m = compute_metrics(y_train, ytr)
        test_m = compute_metrics(y_test, yte)
        print(f"{model_name}: {train_m['MSE']:.3f}    {train_m['MAE']} {train_m['R^2']:.3f}       {test_m['MSE']:.3f}  {test_m['MAE']} {test_m['R^2']:.3f}")

Table 1: Predicting Horse Power using Fuel Economy (MPG)
Model             Train MSE  Train MAE          Train R^2 | Test MSE Test MAE           Test R^2
Linear Reg      : 357.699    16.061689166201514 0.906       318.561  14.940628099855125 0.913
Poly Reg (deg 2): 350.880    15.995824272236028 0.908       331.105  15.148329708541159 0.909
Poly Reg (deg 3): 345.109    15.746761759031665 0.910       318.404  14.764973157076463 0.913
Poly Reg (deg 4): 339.700    15.508464872230821 0.911       313.799  14.735471434136919 0.914


### 1.5 Discussion and interpretation

#### Which model performs best on the test set and why?
The model that performed best on the test set was the polynomial regression model with a degree of 4 since it has the lowest test mean squared error, the lowest test mean absolute error, and the highest R^2 value.

#### Does increasing polynomial degree always improve performance? If not, explain what you observe.
Not necessarily. In the test metrics, we see that going from a linear regression model to a polynomial model with degree two actually increased both error metrics and lowered the R^2 value, which means that increasing the polynomial degree doesn't always improve performance.

#### If a model performs unexpectedly poorly (e.g., low R2 or large test error), propose at least two plausible reasons, such as:
All models perform well since they all have an R^2 value of over 0.9

### 2.1 Load and inspect the dataset

In [ ]:

# ============================================================
# Load dataset
# ============================================================

DATA_PATH = "electricity_consumption_based_weather_dataset.csv"
df = pd.read_csv(DATA_PATH)

print("Shape:", df.shape)
print("\nColumns:")
print(df.columns.tolist())

display(df.head())

print("\nSummary statistics:")
display(df.describe(include="all"))

print("\nMissing values per column:")
display(df.isna().sum())


Shape: (1433, 6)

Columns:
['date', 'AWND', 'PRCP', 'TMAX', 'TMIN', 'daily_consumption']


,date,AWND,PRCP,TMAX,TMIN,daily_consumption
0,2006-12-16,2.5,0.0,10.6,5.0,1209.176
1,2006-12-17,2.6,0.0,13.3,5.6,3390.460
2,2006-12-18,2.4,0.0,15.0,6.7,2203.826
3,2006-12-19,2.4,0.0,7.2,2.2,1666.194
4,2006-12-20,2.4,0.0,7.2,1.1,2225.748



Summary statistics:


,date,AWND,PRCP,TMAX,TMIN,daily_consumption
count,1433,1418.000000,1433.000000,1433.000000,1433.000000,1433.000000
unique,1433,NaN,NaN,NaN,NaN,NaN
top,2010-11-26,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN
mean,NaN,2.642313,3.800488,17.187509,9.141242,1561.078061
std,NaN,1.140021,10.973436,10.136415,9.028417,606.819667
min,NaN,0.000000,0.000000,-8.900000,-14.400000,14.218000
25%,NaN,1.800000,0.000000,8.900000,2.200000,1165.700000
50%,NaN,2.400000,0.000000,17.800000,9.400000,1542.650000
75%,NaN,3.300000,1.300000,26.100000,17.200000,1893.608000



Missing values per column:


,0
date,0
AWND,15
PRCP,0
TMAX,0
TMIN,0
daily_consumption,0


### 2.2 Train/test Split

In [ ]:
# Randomly split the dataset into 70% training and 30% testing.
# Use a fixed random state for reproducibility -- seed=42.

TARGET_COL = "daily_consumption"
EXTRA_COL = "date"

df_no_date = df.drop(columns=[EXTRA_COL]).copy() # drop the date column due to formatting inconsistencies

X, y = prepare_xy(df_no_date, TARGET_COL) # handles missing values here by removing rows with missing values

x_train, x_test, y_train, y_test = split_data(X, y) # default 30% testing


### 2.3 Model training: Linear + Polynomial regression

In [ ]:
# Linear Regression
ln_model = LinearRegression()

ln_model.fit(x_train, y_train)

yhat_train_ln = ln_model.predict(x_train)
yhat_test_ln = ln_model.predict(x_test)


# Degree 2
deg_two_model = Pipeline([
                ("poly", PolynomialFeatures(degree=2, include_bias=False)),
                ("lr", LinearRegression())])

deg_two_model.fit(x_train, y_train)

yhat_train_two = deg_two_model.predict(x_train)
yhat_test_two = deg_two_model.predict(x_test)

# Degree 3
deg_three_model = Pipeline([
                ("poly", PolynomialFeatures(degree=3, include_bias=False)),
                ("lr", LinearRegression())])

deg_three_model.fit(x_train, y_train)

yhat_train_three = deg_three_model.predict(x_train)
yhat_test_three = deg_three_model.predict(x_test)

# Degree 4
deg_four_model = Pipeline([
                ("poly", PolynomialFeatures(degree=4, include_bias=False)),
                ("lr", LinearRegression())])

deg_four_model.fit(x_train, y_train)

yhat_train_four = deg_four_model.predict(x_train)
yhat_test_four = deg_four_model.predict(x_test)

### 2.4 Model evaluation (train and test)

In [ ]:
# Print metrics
print("Table 2: Predicting Electricity Consumption using Weather Data")
print("=================================================================================================")
print("Model             Train MSE     Train MAE          Train R^2 | Test MSE Test MAE           Test R^2")
print("=================================================================================================")
for model_name, ytr, yte in [
    ("Linear Reg      ", yhat_train_ln, yhat_test_ln),
    ("Poly Reg (deg 2)", yhat_train_two, yhat_test_two),
    ("Poly Reg (deg 3)", yhat_train_three, yhat_test_three),
    ("Poly Reg (deg 4)", yhat_train_four, yhat_test_four)
]:
        train_m = compute_metrics(y_train, ytr)
        test_m = compute_metrics(y_test, yte)
        print(f"{model_name}: {train_m['MSE']:.3f}    {train_m['MAE']}   {train_m['R^2']:.3f}       {test_m['MSE']:.3f}  {test_m['MAE']} {test_m['R^2']:.3f}")

Table 2: Predicting Electricity Consumption using FWeather Data
Model             Train MSE     Train MAE          Train R^2 | Test MSE Test MAE           Test R^2
Linear Reg      : 272403.396    384.46501594660754   0.276       248125.786  375.404536584092 0.299
Poly Reg (deg 2): 264765.770    379.6487530529937   0.296       255268.494  379.0390828593324 0.279
Poly Reg (deg 3): 259249.535    375.9529009135978   0.311       265623.658  385.23516692401824 0.250
Poly Reg (deg 4): 251909.339    372.1165656737124   0.330       12151486.443  578.64220053637 -33.314


### 2.5 Discussion and interpretation

#### Which model generalizes best (best test performance), and what does that tell you about the relationship between weather and electricity usage?

The model with that generalizes the best is the linear regression model since it has the lowest test errors (MSE & MAE) and the highest R^2 value. This tells us that weather has an impact on electricity usage. However, none of the models perform well, as seen by the high MSE, MAE, and low R^2 values in all categories.

#### Do polynomial models improve the fit compared to linear regression? If yes, why might electricity consumption have nonlinear dependence on weather?



#### If higher-degree models perform worse on the test set, explain this behavior using evidence from metrics (e.g., train error decreases but test error increases).
The higher degree models do perform worse when comparing taining and test sets (especially looking at the results for the degree 4 model). A potential reason for this is due to overfitting, which is supported by the much lower test R^2 of -33 to the train R^2 of 0.330.

#### If none of the models achieve good test performance, provide at least two reasons supported by your outputs (e.g., limited feature set, high noise, unmodeled drivers such as occupancy/behavior,seasonal effects)
None of the models have good test OR training performance (low MSE/MAE and R^2 values close to 1), which tells us that there is an issue with relationships in the data itself. This could be attributed to having a limited feature set, and thus the model cannot explain the variance in the data (as seen in the low R^2 value). Additionally, there can be a lot of noise in the data which makes it difficult for the model to have accurate predictions which is seen in the high MAE values.